In [1]:
import nltk # using textBlob and tensoflow (CNN and LSTM )
import pprint
import tensorflow as tf
import pandas as pd
import numpy as np
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
# some source for this dic 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\natsu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Load data
data = pd.read_csv('movie_reviews.csv')

In [3]:
# Target distribution
data['label'].value_counts()

label
0    20049
1    19951
Name: count, dtype: int64

In [5]:
# NULL check
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [4]:
data.head(5)

,text,label
0,If you havent seen this movie than you need to...,1
1,but Cinderella gets my vote not only for the w...,0
2,This movie is pretty cheesy but I do give it c...,1
3,I have not seen a Van Damme flick for a while ...,1
4,This is a sleeper It defines Nicholas Cage The...,1


In [11]:
labels = np.array(data['label'].values, dtype=np.int8)
labels

array([1, 0, 1, ..., 1, 0, 0], dtype=int8)

In [12]:
texts = data.text.values

In [15]:
vocab_size = 10000 # 
input_length = 500
embedding_vector_length = 32

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=vocab_size)
# Create vocab
tokenizer.fit_on_texts(texts)

In [21]:
# Encoding all texts in data
encoded_texts = tokenizer.texts_to_sequences(texts)

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_sequence = pad_sequences(encoded_texts, maxlen=input_length) # Padding all sequences to same length

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=input_length))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           320000    
                                                                 
 spatial_dropout1d (Spatial  (None, 500, 32)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 336651 (1.28 MB)
Trainable params: 336651 (1.28 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [23]:
history = model.fit(padded_sequence,labels,validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5


1000/1000 [==============================] - 625s 611ms/step - loss: 0.4620 - accuracy: 0.7852 - val_loss: 0.3404 - val_accuracy: 0.8536
Epoch 2/5
1000/1000 [==============================] - 620s 620ms/step - loss: 0.3104 - accuracy: 0.8773 - val_loss: 0.2884 - val_accuracy: 0.8896
Epoch 3/5
1000/1000 [==============================] - 560s 560ms/step - loss: 0.2573 - accuracy: 0.9004 - val_loss: 0.2874 - val_accuracy: 0.8867
Epoch 4/5
1000/1000 [==============================] - 256s 256ms/step - loss: 0.2700 - accuracy: 0.8845 - val_loss: 0.3330 - val_accuracy: 0.8654
Epoch 5/5
1000/1000 [==============================] - 248s 248ms/step - loss: 0.2163 - accuracy: 0.9173 - val_loss: 0.3235 - val_accuracy: 0.8765


In [36]:
def model_predict(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=input_length)
    prediction = model.predict(padded_sequence)
    if prediction > 0.5:
        return 1
    else:
        return 0

In [39]:
submission = pd.read_csv('test_data.csv')
submission['Category'] = submission['text'].apply(model_predict)
submission.drop('text', axis=1, inplace=True)
submission.to_csv('submission.csv', index=False)

1/1 [==============================] - 0s 72ms/step
